<a href="https://colab.research.google.com/github/igorvojnyak/datavojnyak.github.io/blob/master/Text_Calssification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install Flask


In [3]:
pip install Flask-RESTful

     |████████████████████████████████| 51kB 2.8MB/s 


In [4]:
pip install pymorphy2

     |████████████████████████████████| 51kB 1.7MB/s 
     |████████████████████████████████| 7.1MB 6.5MB/s 


In [5]:
pip install contractions

     |████████████████████████████████| 317kB 3.6MB/s 
     |████████████████████████████████| 245kB 8.0MB/s 
  Created wheel for pyahocorasick: filename=pyahocorasick-1.4.0-cp36-cp36m-linux_x86_64.whl size=81705 sha256=b5543845d35630f19b2ff4cab1f439485db22ae2c447ec62df9f8b65edecd2d4
  Stored in directory: /root/.cache/pip/wheels/0a/90/61/87a55f5b459792fbb2b7ba6b31721b06ff5cf6bde541b40994
Successfully built pyahocorasick


In [6]:
pip install pymorphy2

In [7]:
pip install pyaspeller

In [8]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
import logging
import pandas as pd
import numpy as np
from numpy import random
from sklearn import datasets
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.externals import joblib
import gensim
import nltk
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
import re
from bs4 import BeautifulSoup
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import LinearSVC
from nltk.stem.snowball import SnowballStemmer

In [ ]:
train = pd.read_csv('/content/Тестовое задание NLP - Training data.csv')
test = pd.read_csv('/content/Тестовое задание NLP - Test data.csv')
#df1 = pd.read_csv('/content/Тестовое задание NLP - Test data.csv')
other_test = {'Пример текста': ['Кажется, ты сделал грубую ошибку.','Здесь очень быстро темнеет.','Говорят, зима будет холодной.','По-настоящему можно видеть только сердцем','Я прочитал письмо','Моя машина не заводится.','Ты пришел очень быстро']}
other_train = {'Пример текста': ['Мне очень жаль','Прошу прощения','Простите не могу','Извините я хотел как лучше','Это очень мило с вашей стороны','В любом случае, спасибо','Спасибо заранее','Не стоит благодарности','Могу ли я вам помочь','Все в порядке','Не волнуйтесь об этом','Сюда пожалуйста']}
data = pd.DataFrame(other_train)
data['Класс']='other'
train = pd.concat([train, data], ignore_index=True)
data = pd.DataFrame(other_test)
data['Класс']='other'
test = pd.concat([test, data], ignore_index=True)
train_text = train['Пример текста']
test_text = test['Пример текста']
all_text = pd.concat([train, test], ignore_index=True)

In [ ]:
speller = YandexSpeller(ignore_tags=True, ignore_digits=True)

In [ ]:
text=[w for w in test['Пример текста']]
changes = {change['word']: change['s'][0] for change in speller.spell(text)}
for word, suggestion in changes.items():
    text = [ item.replace(word, suggestion) for item in text]
test['Пример текста'] = pd.DataFrame(text)

In [ ]:
text=[w for w in train['Пример текста']]
changes = {change['word']: change['s'][0] for change in speller.spell(text)}
for word, suggestion in changes.items():
    text = [ item.replace(word, suggestion) for item in text]
train['Пример текста'] = pd.DataFrame(text)

In [ ]:
import re 
r =  re.compile("[а-яА-Я]+")
def preprocessing(line):
    line = line.lower()
    line = re.sub(r"[{}]".format(string.punctuation), " ", line)
    return line
def remove_num(text):
    return(''.join(ch for ch in text if not ch.isdigit()))
def remove_special_characters(text, remove_digits=False):
    pattern = r'[^а-яА-Я+0-9\s]' if not remove_digits else r'[^а-яА-Я+\s]'
    text = re.sub(pattern, '', text)
    return text
def tokenization_w(words):
    w_new = []
    for w in (words[:][0]):  # for NumPy = words[:]
        w_token = word_tokenize(w)
        if w_token != '':
            w_new.append(w_token)
    return w_new

In [ ]:
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.snowball import RussianStemmer
stemmer = RussianStemmer(False)

train['Пример текста'] = train['Пример текста'].apply(remove_special_characters)
train['Пример текста']=train['Пример текста'].apply(lambda x: ''.join([stemmer.stem(word) for word in x]))
train['Пример текста'] = [w for w in filter(r.match, train['Пример текста'])]
train['Пример текста']= train['Пример текста'].apply(preprocessing)
train = train.copy()
train['Пример текста'] =train['Пример текста'].apply(lambda x: ' '.join([word for word in r.findall(x.lower())]))
train['Пример текста'] =  train['Пример текста'].apply(lambda x:  ' '.join([contractions.fix(word) for word in x.split()]))
train['Пример текста'] = train['Пример текста'].apply(remove_num)
train['Пример текста'] =  train['Пример текста'].apply(lambda x: ''.join([word for word in x if word not in punctuation]))

In [ ]:
test['Пример текста'] = test['Пример текста'].apply(remove_special_characters)
test['Пример текста']=test['Пример текста'].apply(lambda x: ''.join([stemmer.stem(word) for word in x]))
test['Пример текста'] = [w for w in filter(r.match, test['Пример текста'])]
test['Пример текста']= test['Пример текста'].apply(preprocessing)
test = test.copy()
test['Пример текста'] =test['Пример текста'].apply(lambda x: ' '.join([word for word in r.findall(x.lower())]))
test['Пример текста'] =  test['Пример текста'].apply(lambda x:  ' '.join([contractions.fix(word) for word in x.split()]))
test['Пример текста'] = test['Пример текста'].apply(remove_num)
test['Пример текста'] =  test['Пример текста'].apply(lambda x: ''.join([word for word in x if word not in punctuation]))

In [ ]:
train['Класс_id'] = train['Класс'].factorize()[0]
category_id_df = train[['Пример текста', 'Класс_id']].drop_duplicates()
test['Класс_id'] = test['Класс'].factorize()[0]
category_id_df = test[['Пример текста', 'Класс_id']].drop_duplicates()

In [ ]:
import nltk
from nltk.tokenize import word_tokenize

In [ ]:
X_train= train['Пример текста']
test_X = test['Пример текста']
y_train = train['Класс'] 
test_y = test['Класс']

In [ ]:
from sklearn.linear_model import SGDClassifier
sgd = Pipeline([('vect', CountVectorizer()),
                ('tfidf', TfidfTransformer()),
                ('clf', SGDClassifier(loss='hinge', penalty='l1',alpha=1e-3, random_state=42, max_iter=5, tol=None)),])
sgd.fit(X_train, y_train)
y_pred = sgd.predict(test_X)

print('accuracy %s' % accuracy_score(y_pred, test_y))

In [ ]:
pickle.dump(sgd, open('final_prediction.pickle', 'wb'))

In [ ]:
from flask import Flask, request, redirect, url_for, flash, jsonify
import numpy as np
import pickle as p
import json


app = Flask(__name__)


@app.route('/api/', methods=['POST'])
def makecalc():
    data = request.get_json()
    prediction = np.array2string(model.predict(data))

    return jsonify(prediction)

if __name__ == '__main__':
    modelfile = '/content/final_prediction.pickle'
    model = p.load(open(modelfile, 'rb'))
    app.run(debug=True, host='0.0.0.0')